In [21]:
from django.db.models import OuterRef, Subquery
from django.db.models import F
from public_data.models import Ocsge2015, CouvertureSol, UsageSol, Artificielle2018
from django.db import connection

In [42]:
# update avec une sous requête
code_couverture = 'couverture'
code_usage = None

label_couv = CouvertureSol.objects.filter(code_prefix=OuterRef(code_couverture))
Artificielle2018.objects.all().update(couverture_label=Subquery(label.values('label')[:1]))
connection.queries.pop()

{'sql': 'UPDATE "public_data_artificielle2018" SET "couverture_label" = (SELECT U0."label" FROM "public_data_couverturesol" U0 WHERE U0."code_prefix" = "public_data_artificielle2018"."couverture" LIMIT 1)',
 'time': '0.143'}

In [40]:
Artificielle2018.objects.all().update(couverture_label="")

18104

In [32]:
# test de l'utilisation de extra pour faire une jointure
qs = Artificielle2018.objects.all()
table_name = Artificielle2018._meta.db_table
kwargs = {
    "select": {"couverture_label": "public_data_couverturesol.label"},
    "tables": ["public_data_couverturesol"],
    "where": [f"public_data_couverturesol.code_prefix = {table_name}.couverture"]
}
qs = qs.extra(**kwargs)
print(qs.query)

SELECT (public_data_couverturesol.label) AS "couverture_label", "public_data_artificielle2018"."id", "public_data_artificielle2018"."couverture", "public_data_artificielle2018"."surface", "public_data_artificielle2018".couverture_label, "public_data_artificielle2018"."usage_label", "public_data_artificielle2018"."mpoly"::bytea FROM "public_data_artificielle2018" , "public_data_couverturesol" WHERE (public_data_couverturesol.code_prefix = public_data_artificielle2018.couverture)


In [33]:
# executer une raw query pour faire un update
update_query = """
UPDATE public_data_artificielle2018 a
SET couverture_label = c.label
FROM 
    public_data_couverturesol c
WHERE 
    c.code_prefix = a.couverture
"""

with connection.cursor() as cursor:
    cursor.execute(update_query)

In [37]:
for o in Artificielle2018.objects.all().values("couverture", "couverture_label").distinct()[:10]:
    print(o["couverture"], o["couverture_label"])

CS1.1 Surfaces anthropisées


In [38]:
with connection.cursor() as cursor:
    cursor.execute(update_query)